In [5]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

import pickle
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go

from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve
from sklearn.metrics import classification_report

In [2]:
full_test = pd.read_csv("../../data/split-data/with-mhc/test/full_test.csv")
# full_test

,CDR3b,epitope,MHC,binder
0,ASSEWVAGVQETQY,KLGGALQAK,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,0
1,ASSSTVYNEQF,VTEHDTLLY,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,0
2,ASSRRHFSVATLMNTEAF,RIAAWMATY,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,0
3,ASSQGGSDNEQF,TPRVTGGGAM,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,0
4,ASSQRDRGSDTQY,QPRAPIRPI,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,0
...,...,...,...,...
2893481,ASSFPTEGLGATDTQY,IMDQVPFSV,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,0
2893482,ASSRQDSGANVLT,YLNDHLEPWI,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,0
2893483,ASSLSFLGRETQY,AYSSAGASI,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFSTSVSRPGRGEPRF...,0
2893484,ASSPGLAGPTNTGELF,SLFNTVATLY,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,0


In [3]:
def split_dataframe_by_position(df, splits):
    dataframes = []
    index_to_split = len(df) // splits
    start = 0
    end = index_to_split
    for split in range(splits):
        temporary_df = df.iloc[start:end, :]
        dataframes.append(temporary_df)
        start += index_to_split
        end += index_to_split
    return dataframes

sp_data = split_dataframe_by_position(full_test, 16)

In [ ]:
test01 = sp_data[0]
test02 = sp_data[1]
test03 = sp_data[2]
test04 = sp_data[3]
test05 = sp_data[4]
test06 = sp_data[5]
test07 = sp_data[6]
test08 = sp_data[7]
test09 = sp_data[8]
test10 = sp_data[9]
test11 = sp_data[10]
test12 = sp_data[11]
test13 = sp_data[12]
test14 = sp_data[13]
test15 = sp_data[14]
test16 = sp_data[15]
test17 = full_test.loc[2893472:2893485, :]

test01 = test01.reset_index(drop=True)
test02 = test02.reset_index(drop=True)
test03 = test03.reset_index(drop=True)
test04 = test04.reset_index(drop=True)
test05 = test05.reset_index(drop=True)
test06 = test06.reset_index(drop=True)
test07 = test07.reset_index(drop=True)
test08 = test08.reset_index(drop=True)
test09 = test09.reset_index(drop=True)
test10 = test10.reset_index(drop=True)
test11 = test11.reset_index(drop=True)
test12 = test12.reset_index(drop=True)
test13 = test13.reset_index(drop=True)
test14 = test14.reset_index(drop=True)
test15 = test15.reset_index(drop=True)
test16 = test16.reset_index(drop=True)
test17 = test17.reset_index(drop=True)

In [ ]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
        
def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))    
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)    
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    e = a.values.flatten()
    return e

def data_representation_mhc(X_data):
    seq_cdr3_lst = X_data['CDR3b'].to_list()
    seq_epitope_lst = X_data['epitope'].to_list()
    seq_mhc_lst = X_data['MHC'].to_list()
    
    lst1 = []
    for seq in seq_cdr3_lst:
        e=one_hot_encode(seq)
        m_cdr3 = np.reshape(e, (-1, 20))
        padding = 19 - len(m_cdr3)
        c = np.concatenate((m_cdr3, np.zeros(padding * 20).reshape(padding, 20)), axis=0)
        lst1.append(c)

    lst2 = []
    for seq in seq_epitope_lst:
        e=one_hot_encode(seq)
        m_epitope = np.reshape(e, (-1, 20))
        padding = 11 - len(m_epitope)
        c = np.concatenate((m_epitope, np.zeros(padding * 20).reshape(padding, 20)), axis=0)
        lst2.append(c)
    
    lst3 = []
    for seq in seq_mhc_lst:
        e=one_hot_encode(seq)
        m_mhc = np.reshape(e, (-1, 20))
        padding = 366 - len(m_mhc)
        c = np.concatenate((m_mhc, np.zeros(padding * 20).reshape(padding, 20)), axis=0)
        lst3.append(c)

    res1 = list(map(lambda x: lst1[x].flatten(), range(len(lst1)))) 
    res2 = list(map(lambda x: lst2[x].flatten(), range(len(lst2))))
    res3 = list(map(lambda x: lst3[x].flatten(), range(len(lst3))))
    
    df_res1 = pd.DataFrame(res1)
    df_res2 = pd.DataFrame(res2)
    df_res3 = pd.DataFrame(res3)
    result = pd.concat([df_res1, df_res2, df_res3], axis=1)
    
    return result

In [ ]:
py_test01 = test01.iloc[:, lambda test01: [3]]
py_test02 = test02.iloc[:, lambda test02: [3]]
py_test03 = test03.iloc[:, lambda test03: [3]]
py_test04 = test04.iloc[:, lambda test04: [3]]
py_test05 = test05.iloc[:, lambda test05: [3]]
py_test06 = test06.iloc[:, lambda test06: [3]]
py_test07 = test07.iloc[:, lambda test07: [3]]
py_test08 = test08.iloc[:, lambda test08: [3]]
py_test09 = test09.iloc[:, lambda test09: [3]]
py_test10 = test10.iloc[:, lambda test10: [3]]
py_test11 = test11.iloc[:, lambda test11: [3]]
py_test12 = test12.iloc[:, lambda test12: [3]]
py_test13 = test13.iloc[:, lambda test13: [3]]
py_test14 = test14.iloc[:, lambda test14: [3]]
py_test15 = test15.iloc[:, lambda test15: [3]]
py_test16 = test16.iloc[:, lambda test16: [3]]
py_test17 = test17.iloc[:, lambda test17: [3]]

pX_test01 = data_representation_mhc(test01)
pX_test02 = data_representation_mhc(test02)
pX_test03 = data_representation_mhc(test03)
pX_test04 = data_representation_mhc(test04)
pX_test05 = data_representation_mhc(test05)
pX_test06 = data_representation_mhc(test06)
pX_test07 = data_representation_mhc(test07)
pX_test08 = data_representation_mhc(test08)
pX_test09 = data_representation_mhc(test09)
pX_test10 = data_representation_mhc(test10)
pX_test11 = data_representation_mhc(test11)
pX_test12 = data_representation_mhc(test12)
pX_test13 = data_representation_mhc(test13)
pX_test14 = data_representation_mhc(test14)
pX_test15 = data_representation_mhc(test15)
pX_test16 = data_representation_mhc(test16)
pX_test17 = data_representation_mhc(test17)

pX_test = pd.concat([pX_test01, pX_test02, pX_test03, pX_test04, pX_test05, 
                     pX_test06, pX_test07, pX_test08, pX_test09, pX_test10, 
                     pX_test11, pX_test12, pX_test13, pX_test14, pX_test15, 
                     pX_test16, pX_test17], axis = 0)

py_test = pd.concat([py_test01, py_test02, py_test03, py_test04, py_test05, 
                     py_test06, py_test07, py_test08, py_test09, py_test10, 
                     py_test11, py_test12, py_test13, py_test14, py_test15, py_test16, py_test17])

In [4]:
pX_res = pd.read_csv("../../data/convert-data/with-mhc/res/X.csv")
py_res = pd.read_csv("../../data/convert-data/with-mhc/res/X.csv")

In [ ]:
lst_models = [ ('Random Forest', RandomForestClassifier(bootstrap=False, max_depth=90, max_features=5,
                         n_estimators=400, n_jobs=-1, random_state=42))]

class epitcrModel:
    def __init__(self, pmodel, pX, py):
        self.model = pmodel
        self.model.fit(pX, py)
    
    def predict(self, pnew_data):
        yhat_class = self.model.predict(pnew_data)
        return yhat_class 

    def predict_proba(self, pnew_data):
        yhat_class = self.model.predict_proba(pnew_data)
        return yhat_class 

def saveByPickle(object, path):
    pickle.dump(object, open(path, "wb"))
    print(f"{object} has been saved at {path}.")

rf_tcr = lst_models[0][1]
model_rf = epitcrModel(rf_tcr, pX_res, np.ravel(py_res))

y_rf_test_proba = model_rf.predict_proba(pX_test)
df_test = pd.DataFrame(data = y_rf_test_proba, columns = ["tmp", "predict_proba"])

In [ ]:
df_test = df_test.iloc[:, 1:]
df_prob_test = pd.concat([full_test, df_test], axis=1)
df_prob_test['binder_pred'] = np.where(df_prob_test['predict_proba'] >= 0.5, 1, 0)

df_prob_test.to_csv("../../data/predict-data/full-testset/with-mhc.csv", index=False)